# 模型融合(Ensemble Learning)技术

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20221101183239483.png" alt="image-20221101183239483" style="zoom:50%;" />

In [17]:
import joblib
import pathlib
import warnings
import time

import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

PROCESSED_DATA_DIR = pathlib.Path("../dataset/processed")
BASEMODEL_DIR = pathlib.Path("../app/models/base-models")
EXTERNAL_DIR = pathlib.Path("../dataset/external")

## 加载数据

### 加载数据集

In [18]:
from sklearn.model_selection import train_test_split

inputs = joblib.load(PROCESSED_DATA_DIR / "inputs.joblib")
target = joblib.load(PROCESSED_DATA_DIR / "target.joblib")
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.25, random_state=42, stratify=target)
y_train.value_counts()

fraudulent
0    12632
1     8452
Name: count, dtype: int64

### 加载预处理器

In [19]:
preprocessor = joblib.load(PROCESSED_DATA_DIR / "preprocessor.joblib")
preprocessor

ColumnTransformer(n_jobs=-1,
                  transformers=[('TfidfVectorizer',
                                 TfidfVectorizer(max_df=0.4, max_features=800,
                                                 min_df=0.1, ngram_range=(1, 2),
                                                 stop_words=['ｃ］', '㈧', '不可抗拒',
                                                             'leg', '豁然',
                                                             'himself', '个别',
                                                             '着呢', 'plant',
                                                             '怎么', '不单', '但愿',
                                                             '从优', 'pretty',
                                                             '［①⑧］', 'election',
                                                             '通过', '如次', 'road',
                                                             '尔', '勃然', '上去',
                                                             '促进', '较比', '/',
                                                             '长线', 'walshmouth',
                                                             'sit',
                                                             'discussion',
                                                             'office', ...],
                                                 sublin...
                                                 token_pattern='(?u)\\b[a-zA-Z]{2,}\\b'),
                                 'text'),
                                ('OrdinalEncoder',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['required_education', 'required_experience']),
                                ('OneHotEncoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['employment_type', 'function', 'location',
                                  'department', 'industry']),
                                ('MinMaxScaler', MinMaxScaler(),
                                 ['salary', 'text_length'])])

### 加载基础模型与得分

In [20]:
tree = joblib.load(BASEMODEL_DIR / "DecisionTreeClassifier.joblib")
lr = joblib.load(BASEMODEL_DIR / "LogisticRegression.joblib")
rf = joblib.load(BASEMODEL_DIR / "RandomForestClassifier.joblib")
results = pd.read_json(EXTERNAL_DIR / "model_results.json")
results

,"('Model', '')","('ROC', 'Train')","('ROC', 'Test')","('F1', 'Train')","('F1', 'Test')","('Accuracy', 'Train')","('Accuracy', 'Test')","('Precision', 'Train')","('Precision', 'Test')","('Recall', 'Train')","('Recall', 'Test')"
0,SGDClassifier,0.995117,0.992556,0.956452,0.956184,0.966029,0.965855,0.983818,0.984492,0.930568,0.929459
1,DecisionTreeClassifier,0.971233,0.969964,0.950020,0.948331,0.961183,0.959808,0.981777,0.978302,0.920253,0.920142
2,LogisticRegression,0.996540,0.993322,0.964286,0.960326,0.971899,0.968878,0.982949,0.981919,0.946318,0.939663
3,KNeighborsClassifier,0.998807,0.990524,0.976742,0.968478,0.981503,0.974924,0.984780,0.976115,0.968833,0.960958
4,SVC,0.992185,0.989884,0.947806,0.948582,0.959760,0.960341,0.987264,0.987518,0.911380,0.912600
5,RandomForestClassifier,1.000000,0.996989,1.000000,0.971054,1.000000,0.977236,1.000000,0.990314,1.000000,0.952529
6,AdaBoostClassifier,0.987738,0.986367,0.934950,0.937703,0.950156,0.952339,0.980406,0.984863,0.893523,0.894854
7,GradientBoostingClassifier,0.996043,0.993228,0.964391,0.959654,0.972076,0.968522,0.986543,0.986873,0.943212,0.933895
8,BaggingClassifier,0.999987,0.991447,0.996105,0.972034,0.996888,0.977948,0.999330,0.988532,0.992902,0.956078
9,XGBClassifier,0.999794,0.996277,0.989423,0.973435,0.991552,0.979015,0.993183,0.988117,0.985692,0.959184


## 投票法(Voting)

首先如果模型最终输出的是类别判别结果，则可以通过投票法进行模型融合，投票法会根据**少数服从多数**的规则进行结果输出，例如现有A、B、C、D、E五个模型对现有数据进行预测，结果如下：

<table>
  <tr>
    <th rowspan="2">样本</th>
    <th colspan="5">单模预测结果</th>
    <th colspan="2">投票结果</th>
    <th>最终预测结果</th>
  </tr>
  <tr>
    <th>模型A</th>
    <th>模型B</th>
    <th>模型C</th>
    <th>模型D</th>
    <th>模型E</th>
    <th>预测为0</th>
    <th>预测为1</th>
    <th>规则：少数服从多数</th>
  </tr>
  <tr>
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>0</td>
    <td>1</td>
    <td>4</td>
    <td>1</td>
  </tr>
  <tr>
    <td>2</td>
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>0</td>
    <td>0</td>
    <td>2</td>
    <td>3</td>
    <td>1</td>
  </tr>
  <tr>
    <td>3</td>
    <td>1</td>
    <td>1</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
    <td>3</td>
    <td>2</td>
    <td>0</td>
  </tr>
  <tr>
    <td>4</td>
    <td>1</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
    <td>0</td>
    <td>4</td>
    <td>1</td>
    <td>0</td>
  </tr>
</table>

&emsp;&emsp;而这样的一个投票集成的过程，到底能带来多少性能上的提升呢？从理论上来说，根据[Narasimhamurthy,2003]()研究表明，在多样性构建的比较好的情况下，投票融合性能边界如下：

<center><img src="https://s2.loli.net/2022/05/20/uVrzS79LaBsJgKp.png" alt="image-20220520122231347" style="zoom:33%;" />

能够看出，在单体分类器准确率为80%左右（较为普遍的情况）时，模型投票融合能有平均约15%的准确率提升。当然，该理论实际上是基于分类器相互独立的假设推导而来，而在大多数真实场景下，该假设并不成立，因此该理论的结论可以视作一个理论上限，并不能代表一般情况。

### 硬投票(Hard Voting)

In [21]:
from sklearn.ensemble import VotingClassifier

estimators = [
    ("tree", tree),
    ("lr", lr),
    ("rf", rf),
]
hard_vc = VotingClassifier(estimators, voting="hard")
start_time = time.time()
hard_vc.fit(X_train, y_train)
end_time = time.time()
print(f"VotingClassifier 训练花费的时间为: {(end_time-start_time):.4f} 秒")

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.1s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.1s
[Pipeline] ..... (step 2 of 2) Processing SGDClassifier, total=   0.2s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.2s
[Pipeline]  (step 2 of 2) Processing RandomForestClassifier, total=   3.9s
VotingClassifier 训练花费的时间为: 16.5908 秒


In [22]:
from sklearn.metrics import cohen_kappa_score, recall_score, roc_auc_score

cohen_kappa_score(y_test, hard_vc.predict(X_test))
recall_score(y_test, hard_vc.predict(X_test))
roc_auc_score(y_test, hard_vc.predict(X_test))

np.float64(0.9650872979770613)

### 软投票(Soft Voting)

In [23]:
soft_vc = VotingClassifier(estimators, voting="soft")
start_time = time.time()
soft_vc.fit(X_train, y_train)
end_time = time.time()
print(f"VotingClassifier 训练花费的时间为: {(end_time-start_time):.4f} 秒")

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.1s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.3s
[Pipeline] ..... (step 2 of 2) Processing SGDClassifier, total=   0.2s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.2s
[Pipeline]  (step 2 of 2) Processing RandomForestClassifier, total=   3.8s
VotingClassifier 训练花费的时间为: 16.7782 秒


In [24]:
cohen_kappa_score(y_test, soft_vc.predict(X_test))
recall_score(y_test, soft_vc.predict(X_test))
roc_auc_score(y_test, soft_vc.predict(X_test))

np.float64(0.9649166087871184)

## 均值法

### 权重设计策略

#### 平均为主，博采众长

In [25]:
weight1 = tree.score(X_train, y_train)
weight2 = lr.score(X_train, y_train)
weight3 = rf.score(X_train, y_train)
weights = [weight1, weight2, weight3]
weights

[0.9611079491557579, 0.9718744071333713, 1.0]

In [26]:
soft_vc_weight = VotingClassifier(
    estimators=estimators, 
    voting='soft', 
    weights=weights
)
start_time = time.time()
soft_vc_weight.fit(X_train, y_train)
end_time = time.time()
print(f"VotingClassifier 训练花费的时间为: {(end_time-start_time):.4f} 秒")

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.0s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.0s
[Pipeline] ..... (step 2 of 2) Processing SGDClassifier, total=   0.2s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.0s
[Pipeline]  (step 2 of 2) Processing RandomForestClassifier, total=   3.9s
VotingClassifier 训练花费的时间为: 16.1929 秒


In [27]:
cohen_kappa_score(y_test, soft_vc_weight.predict(X_test))
recall_score(y_test, soft_vc_weight.predict(X_test))
roc_auc_score(y_test, soft_vc_weight.predict(X_test))

np.float64(0.96544890119308)

#### 设计核心评估器与辅助评估器

In [28]:
# 由于随机森林模型在分类中表现最佳，因此设置其为核心评估器，其他两个分类模型为辅助评估器
# 核心评估器的权重设置为100，其他两个分类器的权重设置为1
weight1 = 1
weight2 = 1
weight3 = 100

weights = [weight1, weight2, weight3]
soft_vc_core_weight = VotingClassifier(
    estimators=estimators, 
    voting='soft', 
    weights=weights
)
start_time = time.time()
soft_vc_core_weight.fit(X_train, y_train)
end_time = time.time()
print(f"VotingClassifier 训练花费的时间为: {(end_time-start_time):.4f} 秒")

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.2s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.0s
[Pipeline] ..... (step 2 of 2) Processing SGDClassifier, total=   0.2s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.0s
[Pipeline]  (step 2 of 2) Processing RandomForestClassifier, total=   3.8s
VotingClassifier 训练花费的时间为: 16.3970 秒


In [29]:
cohen_kappa_score(y_test, soft_vc_core_weight.predict(X_test))
recall_score(y_test, soft_vc_core_weight.predict(X_test))
roc_auc_score(y_test, soft_vc_core_weight.predict(X_test))

np.float64(0.97195245006193)

## Stacking法

#### 手动实现StackingClassifier

构造二级学习器的训练数据

In [30]:
train_pre1 = estimators[0][1].predict(X_train)
train_pre2 = estimators[1][1].predict(X_train)
train_pre3 = estimators[2][1].predict(X_train)
train_stack = np.vstack([[train_pre1], [train_pre2], [train_pre3]]).T
test_pre1 = estimators[0][1].predict(X_test)
test_pre2 = estimators[1][1].predict(X_test)
test_pre3 = estimators[2][1].predict(X_test)
test_stack = np.vstack([[test_pre1],[test_pre2], [test_pre3]]).T

In [31]:
from sklearn.linear_model import LogisticRegression

stacking_model = LogisticRegression(random_state=42)
start_time = time.time()
stacking_model.fit(train_stack, y_train)
end_time = time.time()
print(f"VotingClassifier 训练花费的时间为: {(end_time-start_time):.4f} 秒")

VotingClassifier 训练花费的时间为: 0.0136 秒


In [32]:
cohen_kappa_score(y_test, stacking_model.predict(test_stack))
recall_score(y_test, stacking_model.predict(test_stack))
roc_auc_score(y_test, stacking_model.predict(test_stack))

np.float64(0.9785214347026839)

#### sklearn实现

StackingClassifier的核心参数解释如下：

|参数|解释|
|:--:|:--:|
|estimators|一级评估器|
|final_estimator|二级评估器，默认是逻辑回归|
|cv|一级评估器基交叉训练折数|
|stack_method|选择概率结果还是类别结果进行元学习器的训练|
|passthrough|是否额外带入原始数据特征进行元学习器的训练|

* `estimators`参数:参数结构和投票法评估器结构一致，都是需要创建一个由（模型名称、模型）所组成的一个列表。
* `final_estimator`参数：元学习器，只需要实例化一个`sklearn`中的评估器即可；
* `cv`：一级评估器交叉训练的折数，默认是五折。
* `stack_method`：可以选择元学习器的训练数据类型，可选`auto`、`predict_proba`、`decision_function`、`predict`四个不同取值。
    * `predict_proba`: 即带入样本类别概率进行训练，
    * `decision_function`：是SVM特殊的一种模型输出结果，代表样本到分割超平面的（置信）距离，同样也可以充当类似概率的作用，距离越短，则模型判断越不肯定（相当于概率越趋近于0.5）
    * `predict`：则是样本类别结果，相当于是Stacking“硬投票”
    * `auto`：默认参数取值，即根据不同模型，按照`predict_proba`>`decision_function`>`predict`的优先级进行参数选择。当然，对于逻辑回归、决策树和随机森林来说，参数输入`auto`时就是根据预测概率训练元学习器。
* `passthrough`：是否额外带入原始数据集特征进行元学习器训练，
    * 默认参数是`False`：只带入一级学习器的预测结果训练元学习器。
    * `True`：拼凑一个由一级学习器输出结果和原始特征共同拼接而成的数据集，用于元学习器的训练。该操作本质上其实是一种特征增强方法，常常用于层级堆叠结构的模型训练过程，包括某些`Boosting`、深度森林的级联训练等，都有可能用到特征增强技术

##### StackingClassifier硬投票

In [33]:
from sklearn.ensemble import StackingClassifier

hard_vsc = StackingClassifier(
    estimators=estimators, 
    final_estimator=LogisticRegression(random_state=42),
    stack_method="predict",
    cv=5,
)
start_time = time.time()
hard_vsc.fit(X_train, y_train)
end_time = time.time()
print(f"VotingClassifier 训练花费的时间为: {(end_time-start_time):.4f} 秒")

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.2s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.1s
[Pipeline] ..... (step 2 of 2) Processing SGDClassifier, total=   0.3s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.2s
[Pipeline]  (step 2 of 2) Processing RandomForestClassifier, total=   3.8s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.5s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.5s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.6s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.6s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifi

In [34]:
cohen_kappa_score(y_test, hard_vsc.predict(X_test))
recall_score(y_test, hard_vsc.predict(X_test))
roc_auc_score(y_test, hard_vsc.predict(X_test))

np.float64(0.9712960277242053)

##### StackingClassifier软投票

In [35]:
soft_vsc = StackingClassifier(
    estimators=estimators, 
    final_estimator=LogisticRegression(random_state=42), 
    cv=5, 
    stack_method="predict_proba"
)
start_time = time.time()
soft_vsc.fit(X_train, y_train)
end_time = time.time()
print(f"VotingClassifier 训练花费的时间为: {(end_time-start_time):.4f} 秒")

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.0s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.0s
[Pipeline] ..... (step 2 of 2) Processing SGDClassifier, total=   0.2s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.1s
[Pipeline]  (step 2 of 2) Processing RandomForestClassifier, total=   3.8s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.4s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.2s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.2s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   3.4s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifi

In [36]:
cohen_kappa_score(y_test, soft_vsc.predict(X_test))
recall_score(y_test, soft_vsc.predict(X_test))
roc_auc_score(y_test, soft_vsc.predict(X_test))

np.float64(0.9771512441897837)

#### 多层Stacking

其实从此前的介绍不难看出，Stacking的本质就是围绕上一层模型输出结果进行学习，借此提升最终预测效果。而在这个过程中，元学习器本身也是可以输出概率预测结果的，也就是说，某个元学习器之后还可以再堆叠一层元学习器。而如果两层的堆叠能够提升单模效果，那么双层的堆叠则能够进一步提升学习能力，从而进一步提升模型效果。当然，伴随着Stacking结构更加复杂，融合的过拟合风险也会更高

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


tree_final = DecisionTreeClassifier(random_state=42)
rf_final = RandomForestClassifier(random_state=42)
final_layer = StackingClassifier(
    estimators=[("tree_final", tree_final), ("RF_final", rf_final)],
    final_estimator=LogisticRegression(penalty="l1", solver="saga"),
)
multi_layer = StackingClassifier(
    estimators=estimators, final_estimator=final_layer, cv=10
)
start_time = time.time()
multi_layer.fit(X_train, y_train)
end_time = time.time()
print(f"VotingClassifier 训练花费的时间为: {(end_time-start_time):.4f} 秒")

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.2s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.0s
[Pipeline] ..... (step 2 of 2) Processing SGDClassifier, total=   0.2s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.1s
[Pipeline]  (step 2 of 2) Processing RandomForestClassifier, total=   3.9s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   5.8s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.9s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.4s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   0.1s
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.7s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifi

In [38]:
cohen_kappa_score(y_test, multi_layer.predict(X_test))
recall_score(y_test, multi_layer.predict(X_test))
roc_auc_score(y_test, multi_layer.predict(X_test))

np.float64(0.9786799889909474)

#### Stacking模型融合优化策略

`Stacking`模型融合和投票法与均值法类似，原理不难，但要获得一个稳定的优化效果却并没有那么简单。并且相比投票法与均值法，`Stacking`过程采用模型来学习一级评估器的输出结果和标签之间的关系，过拟合的倾向会更加明显。当然，关于`Stacking`容易过拟合的另一个理解的角度是：由于第一层学习器就已经提取了和标签更有关联度的特征，因此元学习器的学习难度偏弱，元学习器更容易过拟合。因此主要的融合优化策略有以下几种：
* 一、二级学习器优化：包括一级学习器训练方法优化与元学习器优化。这是最基础同时也是最核心的优化策略，其核心目的在于平衡Stacking融合的学习效果与过拟合倾向之间的关系
* 多层`Stacking`，通过叠加更多层来提高`Stacking`的学习效果，当然在大多数情况下解决单层`Stacking`融合的过拟合问题已属实不易，要用好多层`Stacking`则更是难上加难
* 特征增强：一级学习器和元学习器带入不同的特征（或衍生特征的）组合，来提高模型多样性，并最终提升Stacking融合效果

In [ ]:
models = {
    "m1": hard_vc,
    "m2": soft_vc,
    "m3":soft_vc_weight,
    "m4": soft_vc_core_weight,
    "m5": stacking_model,
    "m6": hard_vsc,
    "m7": soft_vsc,
    "m8": multi_layer
}

for model_name, model in models.items():
    joblib.dump(
        value=model,
        filename=f"../app/models/base-models/{model_name}.joblib",
        compress=5
    )

## Blending法

> 该模型的训练还未完成，后续进一步完善！

Blending融合的基本过程和Stacking融合较为类似，都是两层模型的基本架构，即都是一级学习器进行训练，然后一级学习器的训练结果带入元学习器进行学习和训练。而和Stacking有所不同的是，为了避免一组数据重复训练导致的过拟合，Blending会在训练集中进一步划分训练集和留出集，一般比例为5：5到9：1不等。其中训练集用于一级学习器的训练，然后一级学习器围绕留出集进行预测，预测结果拼接成类似Stacking中的oof数据集，再将其带入元学习器进行模型训练。至此，即完成了两层模型的训练，其基本过程如下：

### Blending模型的实现

#### 划分训练集和留出集

首先，是训练集和留出集的划分，这里我们按照8：2的比例划分进行划分，并将训练集命名为`trainOE`、留出集命名为`leave`

In [25]:
X_trainOE, X_leave, y_trainOE, y_leave = train_test_split(X_train, y_train,  test_size=0.2, random_state=42)
X_trainOE.sample(n=1, random_state=42)

,location,employment_type,industry,department,telecommuting,has_company_logo,has_questions,required_experience,required_education,function,text,salary,text_length
8787,Hernandezshire,Internship,Finance,Missing,-1.0,-1.0,-1.0,Missing,Missing,Missing,education lecturer couple memory expect intere...,72732.0,216


#### 一级学习器训练

##### 决策树模型

In [30]:
from sklearn.pipeline import Pipeline

tree = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "DecisionTreeClassifier",
            DecisionTreeClassifier(random_state=42)
        ),
    ],
    memory=None,
    verbose=True,
)

tree.fit(X_trainOE, y_trainOE)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   5.3s
[Pipeline]  (step 2 of 2) Processing DecisionTreeClassifier, total=   3.3s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('vectorizer',
                                                  TfidfVectorizer(max_df=0.3,
                                                                  max_features=800,
                                                                  min_df=0.1,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words=['惯常',
                                                                              '每当',
                                                                              '──',
                                                                              '可见',
                                                                              '即或',
                                                                              '她',
                                                                              '⑩',
                                                                              '@',
                                                                              '背地里',
                                                                              '只是',
                                                                              '不得已',
                                                                              '自己',
                                                                              '加强',
                                                                              '满',
                                                                              '先生',
                                                                              '还',
                                                                              '除非',
                                                                              '不够',
                                                                              'empty',
                                                                              '没',
                                                                              '遵照',
                                                                              'hereby',
                                                                              'every',
                                                                              '六',
                                                                              'hundred',
                                                                              '不一',
                                                                              '只怕',
                                                                              'whole',
                                                                              '随著',
                                                                              '假使', ...],
                                                                  sublinear...
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['required_education',
                                                   'required_experience']),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['location', 'department',
                                                   'function',
                                                   'employment_type',
                                                   'industry']),
                                                 ('MinMaxScaler',
  

##### 逻辑回归模型

In [31]:
lr = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "LogisticRegression",
            LogisticRegression(random_state=42)
        ),
    ],
    memory=None,
    verbose=True,
)
lr.fit(X_trainOE, y_trainOE)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   5.4s
[Pipeline]  (step 2 of 2) Processing LogisticRegression, total=   0.3s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('vectorizer',
                                                  TfidfVectorizer(max_df=0.3,
                                                                  max_features=800,
                                                                  min_df=0.1,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words=['惯常',
                                                                              '每当',
                                                                              '──',
                                                                              '可见',
                                                                              '即或',
                                                                              '她',
                                                                              '⑩',
                                                                              '@',
                                                                              '背地里',
                                                                              '只是',
                                                                              '不得已',
                                                                              '自己',
                                                                              '加强',
                                                                              '满',
                                                                              '先生',
                                                                              '还',
                                                                              '除非',
                                                                              '不够',
                                                                              'empty',
                                                                              '没',
                                                                              '遵照',
                                                                              'hereby',
                                                                              'every',
                                                                              '六',
                                                                              'hundred',
                                                                              '不一',
                                                                              '只怕',
                                                                              'whole',
                                                                              '随著',
                                                                              '假使', ...],
                                                                  sublinear...
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['required_education',
                                                   'required_experience']),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['location', 'department',
                                                   'function',
                                                   'employment_type',
                                                   'industry']),
                                                 ('MinMaxScaler',
  

##### 随机森林模型

In [33]:
rf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "RandomForestClassifier",
            RandomForestClassifier(random_state=42)
        ),
    ],
    memory=None,
    verbose=True,
)
rf.fit(X_trainOE, y_trainOE)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   7.8s
[Pipeline]  (step 2 of 2) Processing RandomForestClassifier, total=   4.2s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('vectorizer',
                                                  TfidfVectorizer(max_df=0.3,
                                                                  max_features=800,
                                                                  min_df=0.1,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words=['惯常',
                                                                              '每当',
                                                                              '──',
                                                                              '可见',
                                                                              '即或',
                                                                              '她',
                                                                              '⑩',
                                                                              '@',
                                                                              '背地里',
                                                                              '只是',
                                                                              '不得已',
                                                                              '自己',
                                                                              '加强',
                                                                              '满',
                                                                              '先生',
                                                                              '还',
                                                                              '除非',
                                                                              '不够',
                                                                              'empty',
                                                                              '没',
                                                                              '遵照',
                                                                              'hereby',
                                                                              'every',
                                                                              '六',
                                                                              'hundred',
                                                                              '不一',
                                                                              '只怕',
                                                                              'whole',
                                                                              '随著',
                                                                              '假使', ...],
                                                                  sublinear...
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['required_education',
                                                   'required_experience']),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['location', 'department',
                                                   'function',
                                                   'employment_type',
                                                   'industry']),
                                                 ('MinMaxScaler',
  

#### 元学习器训练和测试数据创建

In [36]:
train_blending = pd.DataFrame(
    {
        "lr_blending": lr.predict_proba(X_leave)[:, 1],
        "rf_blending": rf.predict_proba(X_leave)[:, 1],
        "tree_blending": tree.predict_proba(X_leave)[:, 1],
    }
)
train_blending.sample(5, random_state=42)

,lr_blending,rf_blending,tree_blending
463,0.021907,0.07,0.0
2426,0.999857,1.00,1.0
2661,0.174358,0.12,0.0
1483,0.000082,0.03,0.0
2860,0.026935,0.00,0.0


In [35]:
test_blending = pd.DataFrame(
    {
        "lr_blending": lr.predict_proba(X_test)[:, 1],
        "RF_blending": rf.predict_proba(X_test)[:, 1],
        "tree_blending": tree.predict_proba(X_test)[:, 1],
    }
)
test_blending.sample(5, random_state=42)

,lr_blending,RF_blending,tree_blending
2476,0.012832,0.01,0.0
6770,0.162573,0.11,0.0
4635,0.033287,0.03,0.0
3047,0.007526,0.00,0.0
4092,0.000611,0.02,0.0


#### 元学习器训练测试

In [40]:
lr = LogisticRegression(random_state=42)
lr.fit(train_blending, y_leave)

LogisticRegression(random_state=42)